In [ ]:
import networkx as nx
import pandas as pd
from scipy import stats

In [ ]:
def calculate_career(row):
    start_date, end_date = row['재직기간'].split('~')
    start_year, start_month, _ = start_date.split('.')
    end_year, end_month, _ = end_date.split('.')
    
    career = (int(end_year) - int(start_year)) * 12 + (int(end_month) - int(start_month))
    return career

# 경력 계산 적용
df['career'] = df.apply(calculate_career, axis=1)

In [ ]:
def influence(G, df):
    #영향력 평가지표 추출
    degree_centrality = nx.degree_centrality(G)
    eigenvector_centrality = nx.eigenvector_centrality(G,weight='weight')
    pagerank = nx.pagerank(G, alpha=0.9, weight='weight')
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    closeness_centrality = nx.closeness_centrality(G)
    dicts = [degree_centrality, eigenvector_centrality, pagerank, betweenness_centrality, closeness_centrality]
    merged_dict = {}
    stats.zscore(dicts) #zscore 정규화 (되나?)

    df['career'] = df['career'].apply(stats.zscore) #dataframe정규화
    df['의결권 있는 주식수'] = df['의결권 있는 주식수'].apply(stats.zscore)

    #딕셔너리값 합치기(영향력 평가지표 더하기)
    for dict in dicts:
        for key, value in dict.items():
            if key in merged_dict:
                merged_dict[key] += value
            else:
                merged_dict[key] = value
    
    #경력 더해주기
    for key, value in merged_dict.items():
        merged_dict[key] += df.loc[merged_dict[key],'career']
    
    #의결권있는 주식수 더해주기
    for key, value in merged_dict.items():
        merged_dict[key] += df.loc[merged_dict[key],'의결권 있는 주식수']
    
    return merged_dict

In [ ]:
#특정 인물과 가까운 인물들 rank
def find_closest(G, name, rank=10):  
    neighbors_list = list(G.neighbors(f"{name}"))
    
    temp_dict = {}
    for n in neighbors_list:
        weight = G.edges[f"{name}", n]['weight']
        temp_dict[n] = weight
    
    sorted_list = sorted(temp_dict.items(), key=(lambda x:x[1]), reverse=True)
    for n, w in sorted_list[0:rank]:  
        print(f'{n}:  {w}')

In [ ]:
#인물 중요도 변화

years_G = [G2014, G2015, G2016, G2017, G2018, G2019, G2020, G2021, G2022, G2023] #연도별 network들
G_years = []

#만든 network사용- 연도별로 묶기
for year_G in years_G:
    G_years.append(year_G)


evol = [influence(G) for G in G_years]
 

index = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
degree_evol_df = pd.DataFrame.from_records(evol, index=index).fillna(0) # 모든 연도에 모든 인물이 있는 게 아니므로, N/A는 0으로 채워줌

# 각 연도의 중요도 top 5 인물 선택
set_of_char = set()
for i in index:  
    set_of_char |= set(list(degree_evol_df.T[i].sort_values(ascending=False)[0:5].index)) 
            # |=는 왼쪽과 오른쪽의 합집합을 왼쪽에 assign해주는 역할
list_of_char = list(set_of_char)


degree_evol_df[list_of_char].plot(figsize=(13, 7));